<a href="https://colab.research.google.com/github/shreyapamecha/Room-Allocation-using-Particle-Swarm-Optimization-PSO-/blob/main/Assignment2_ShreyaPamecha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Assignment 2 [NIC] ##
# ROOM ALLOCATION USING PSO #

from collections import defaultdict
import pandas as pd
from random import *
import numpy as np
from random import *
from operator import *

def sortThird(val):
  return val[2]

# Extracting the preference list
preferences = pd.read_excel('/content/drive/My Drive/SEMs-&-E-books/SEM-7/NIC/preference_list.xlsx')

# Total rooms available (single + double)
total_rooms = 40
double_room = []
single_room = []

Gbest_list = [] # Keeps track of the Gbest at every iteration

# separating the double rooms and single rooms
for i in range(0,total_rooms,1):
  if int(preferences._get_value(57,i+1,takeable=True)) == 1:
    single_room.append(i+1)
  else:
    double_room.append(i+1)

print('single_room:', single_room)
print('double_room: ', double_room)

# Extracting only the Preferences
preferences = preferences.iloc[0:56,0:41].astype('int')

# Number of particles 
particles = 100

# Maximum Iteration
max_iter = 2000

# All boys and girls
all = list(range(1,57))

allocat_all = 0

# Dictionary with number 1 to 56 as keys and '0' as their values as no room has been allotted to them yet
room_allocation = dict.fromkeys(all,allocat_all)

# Girls are from 1 to 28
girls = list(range(1,29))
# Boys are from 29 to 56
boys = list(range(29,57))

# 'all_particle' would have positions, velocities, fitness values and Pbest for all the 100 particles
all_particles = []


### Initialization ###
# Position of all the 100 particles (56 dimensions)
# Initializing the rooms for all people 
for j in range(particles):
  # Dictionary with 56 people as keys and room number as their values
  room_allocation = dict.fromkeys(all,allocat_all)
  girls = list(range(1,29))
  boys = list(range(29,57))

  # Starting with alloting double rooms first
  for d_room in double_room:
    drooms_to_gb = [] # double rooms for girls and boys
    b_or_g = randint(0,1) #randomly selecting whether girls should get the nth double room or boys; zero for girls and one for boys

    # Zero for selecting girls
    if b_or_g == 0 and len(girls)>=2: # Highly unlikely chances that all the chosen students for double rooms are girls
      # randomly selecting 2 girls out of the all the remaining girls
      drooms_to_gb = sample(set(girls),2)
      # The selected girls for the nth room are removed from the girls list as they have been allotted a room
      girls = list(set(girls) - set(drooms_to_gb))

      # Now, filling the room allocation dictionary with people as keys and room numbers as values
      for item in drooms_to_gb:
        room_allocation[item] = d_room

    # One for selecting boys
    elif b_or_g == 1 and len(boys)>=2:  
      drooms_to_gb = sample(set(boys),2) # randomly selecting 2 boys out of the all the remaining boys
      # The selected boys for the nth double room are removed from the boys list as they have been allotted a room
      boys = list(set(boys) - set(drooms_to_gb)) 

      # Now, filling the room allocation dictionary with people as keys and room numbers as values
      for item in drooms_to_gb:
        room_allocation[item] = d_room
      # print(boys, d_room)


  # Alloting single rooms to the remaining people
  for s_room in single_room:

    sroom_to_gb = [] #single room for girl or boy

    # if there are people left in both girls' and boys' list
    if len(boys)!=0 and len(girls)!=0:
      b_or_g = randint(0,1) #randomly selecting whether girls should get the nth single room or boys; zero for girls and one for boys
      # Zero for girls
      if b_or_g == 0:
        # randomly selecting 1 girl out of the all the remaining girls
        sroom_to_gb = sample(set(girls),1)
        # The selected girl for the nth single room is removed from the girls' list as she has been allotted a room
        girls = list(set(girls) - set(sroom_to_gb))

        # filling the room allocation dictionary
        for item in sroom_to_gb:
          room_allocation[item] = s_room
      
      else: # One for boys
        # randomly selecting 1 boy out of the all the remaining boys
        sroom_to_gb = sample(set(boys),1)
        # The selected boy for the nth single room is removed from the boys' list as he has been allotted a room
        boys = list(set(boys) - set(sroom_to_gb))

        # filling the room allocation dictionary
        for item in sroom_to_gb:
          room_allocation[item] = s_room


    # if there are people left in both girls' list
    elif len(boys)==0:
      # randomly selecting 1 girl out of the all the remaining girls
      sroom_to_gb = sample(set(girls),1)
      # The selected girl for the nth single room is removed from the girls' list as she has been allotted a room
      girls = list(set(girls) - set(sroom_to_gb))

      # filling the room allocation dictionary
      for item in sroom_to_gb:
        room_allocation[item] = s_room

    # if there are people left in both boys' list
    elif len(girls)==0:
      # randomly selecting 1 boy out of the all the remaining boys
      sroom_to_gb = sample(set(boys),1)
      # The selected boy for the nth single room is removed from the boys' list as he has been allotted a room
      boys = list(set(boys) - set(sroom_to_gb))

    # filling the room allocation dictionary
    for item in sroom_to_gb:
      room_allocation[item] = s_room

  # Room allocation for nth particle is done!! [INITIALIZATION]

  nth_particle = []
  nth_particle.append(list(room_allocation.values())) # Initial Positions
  nth_particle.append([0]*56) # Initial Velocities

  # This matrix consists of the positions and velocities of all the partciles
  all_particles.append(nth_particle)


# Deciphering the fitness values of all the particles
for pos_vel in all_particles:
  which_person = 0
  fitness_value = 0 ## Summation of all the preferences
 
  for pos in pos_vel[0]:
    # summing up all the preferences of the rooms alloted to people
    fitness_value += int(preferences.iloc[which_person,pos])
    which_person += 1

  # putting the fitness value as the third column of the list/matrix 'all_particles'
  pos_vel.append(fitness_value)
 
  #Initially Pbest(i) is the copy of positions(i)
  ## Pbest
  pos_vel.append(pos_vel[0])
  pos_vel.append(pos_vel[1])
  pos_vel.append(pos_vel[2])


girls = list(range(1,29))
boys = list(range(29,57))

all_particles.sort(key=sortThird) ## Sorting the rows on the basis of the fitness value which is in the 3rd column of 'all_particles' matrix

## Gbest
Gbest = []
Gbest.append(all_particles[0][0])
Gbest.append(all_particles[0][1])
Gbest.append(all_particles[0][2])

# yyy = Gbest.copy()
Gbest_pos = []

for i in range(56):
  pos = Gbest[0][i]
  Gbest_pos.append(int(preferences.iloc[i,pos]))

Gbest_fitness = Gbest[2]
Gbest_pos_fit = [Gbest_pos, Gbest_fitness]
Gbest_list.append(Gbest_pos_fit)

print(Gbest)
print(Gbest_list)

single_room: [1, 2, 3, 4, 6, 10, 11, 12, 13, 14, 16, 20, 21, 22, 23, 24, 26, 30, 31, 32, 33, 34, 36, 40]
double_room:  [5, 7, 8, 9, 15, 17, 18, 19, 25, 27, 28, 29, 35, 37, 38, 39]
[[26, 23, 5, 20, 21, 24, 22, 31, 27, 15, 35, 27, 16, 13, 32, 40, 5, 6, 19, 11, 1, 33, 34, 15, 35, 30, 36, 19, 8, 25, 7, 4, 29, 7, 9, 29, 25, 39, 39, 17, 37, 38, 10, 17, 9, 28, 28, 37, 18, 8, 38, 12, 2, 14, 3, 18], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 931]
[[[20, 20, 21, 2, 27, 18, 9, 5, 1, 17, 12, 27, 15, 28, 7, 40, 21, 9, 26, 38, 10, 32, 4, 24, 17, 10, 19, 14, 4, 29, 9, 5, 3, 40, 13, 2, 40, 8, 11, 1, 32, 32, 35, 1, 19, 38, 6, 16, 2, 19, 9, 20, 2, 11, 29, 2], 931]]


In [ ]:
## Steps to follow in every iteration:
# 1. Update the velocities (POS)
# 2. Update the positions (POS)
# 3. Room Allocation (Double room - Girls)
# 4. Room Allocation (Double room - Boys)
# 5. Room Allocation (Single room - Boys)
# 6. Room Allocation (Single room - Girls)
# 7. Miscellaneous
# 8. To fill rest of the double rooms
# 9. To fill rest of the single rooms
# 10. Calculating the fitness value
# 11. Update Pbest, Gbest and Gbest_list  

w = random() # Inertia Coefficient
c1 = random() # Personal Acceleration Coefficient
c2 = random() # Social Acceleration Coefficient


for xxx in range(max_iter):
  for pos_vel in all_particles:
    # print('iter: ', iteration)
    # Updating the velocities (POS)
    list1 = np.multiply(w,pos_vel[1])
    x1 = list(np.array(pos_vel[3]) - np.array(pos_vel[0]))
    list2 = np.multiply(c1*random(),x1)
    x2 = list(np.array(Gbest[0]) - np.array(pos_vel[0]))
    list3 = np.multiply(c2*random(),x2)
    pos_vel[1] = list(np.array(list1) + np.array(list2) + np.array(list3))

    # Updating the positions (POS)
    list5 = [int(s) for s in pos_vel[1]]
    pos_vel[0] = list(map(add, pos_vel[0], list5))

    # Ensuring the range of positions is within [1,40]
    for item in pos_vel[0]:
      if item>total_rooms or item<1:
        item = item%(total_rooms+1)


    ##########

    ### Double Room Allocation
    ## Constraint Satisfied: Ensuring everybody gets a room and a double occupancy room should only be allotted to two boys or two girls
    # Starting the room allocation, based on the updated positions obtained 
    room_allocation = dict.fromkeys(all,allocat_all) #Initializing a dictionary with keys as the room numbers and its values as the list of student(s) allotted to that room
    double_room_dict = defaultdict(list) 
    only_one_girl_in_double_room = [] # if only one is there in double room, fill in here sor that we find their partner
    only_one_boy_in_double_room = []
    double_room_copy = double_room.copy() 
    single_room_copy = single_room.copy()

    girls = list(range(1,29))
    boys = list(range(29,57))


    ### Considering girls before boys for double room allotment and then boys before girls for single room allotment
    ## for GIRLS
    for i in range(28):
      if pos_vel[0][i] in double_room_copy: # Seeing if the new positions obtained for girls have any value for double room, if there is, then appending it into double_room_dict where keys are the room numbers and values are the girls
        double_room_dict[pos_vel[0][i]].append(i+1)

    # double_room_dict
    # 3 cases, I: exact 2 girls obtain the same double room, II: only 1 girl obtains a double room (another needs to come) & III: more than 2 girls want the same double room
    for key in dict(double_room_dict):
      # I. if exact 2 girls obtained the same room number, then simply allott the rooms to them
      if len(double_room_dict[key])==2:
        # Room Allocation
        for person in double_room_dict[key]:
          room_allocation[person]=key
        
        # The selected girls for the nth room are removed from the girls list as they have been allotted a room
        girls = list(set(girls) - set(double_room_dict[key]))
        # The selected room is removed from the double_room_copy as it has been allotted to 2 girls
        double_room_copy = list(set(double_room_copy) - set([key]))

      # II. if only 1 girl obtained a double room number, then simply allott the room to her and randomly allott the 2nd place to any girl left [later]
      elif len(double_room_dict[key])==1:
        # Room Allocation
        for person in double_room_dict[key]:
          room_allocation[person]=key
        
        # The selected girl for the nth room is removed from the girls list as she has been allotted a room
        girls = list(set(girls) - set(double_room_dict[key]))
        # A separate list maintaining the list of double rooms which are only singley filled
        only_one_girl_in_double_room.append(key)
        # The selected room is removed from the double_room_copy  
        double_room_copy = list(set(double_room_copy) - set([key])) 

      # III. if more than 2 girls obtained the same double room number, then simply allott the room to the 1st two girls
      elif len(double_room_dict[key])>2:
        i=0
        for person in double_room_dict[key]:
          room_allocation[person]=key
          i+=1
          if i==2:
            break
        
        # The selected girls for the nth room are removed from the girls list as they have been allotted a room
        girls = list(set(girls) - set(double_room_dict[key][0:2]))
        # The selected room is removed from the double_room_copy as it has been allotted to 2 girls
        double_room_copy = list(set(double_room_copy) - set([key]))


    # for BOYS
    only_one_boy_in_double_room = []
    double_room_dict_b = defaultdict(list) 

    for i in range(28,56,1): # Seeing if the new positions obtained for boys have any value for double rooms, if there is, then appending it into double_room_dict where keys are the room numbers and values are the girls
      if pos_vel[0][i] in double_room_copy:
        double_room_dict_b[pos_vel[0][i]].append(i+1)

    # double_room_dict
    # 3 cases, I: exact 2 boys obtain the same double room, II: only 1 boy obtains a double room (another needs to come) & III: more than 2 boys want the same double room
    for key in dict(double_room_dict_b):
      # I. if exact 2 boys obtain the same room number, then simply allott the rooms to them
      if len(double_room_dict_b[key])==2:
        for person in double_room_dict_b[key]:
          # Room Allocation
          room_allocation[person]=key
        
        # The selected boys for the nth room are removed from the boys list as they have been allotted a room
        boys = list(set(boys) - set(double_room_dict_b[key]))
        # The selected room is removed from the double_room_copy as it has been allotted to 2 boys
        double_room_copy = list(set(double_room_copy) - set([key]))

      # II. if only 1 boy obtain a double room number, then simply allott the room to him and randomly allott the 2nd place to any boy left [later]
      elif len(double_room_dict_b[key])==1:
        for person in double_room_dict_b[key]:
          room_allocation[person]=key
        
        # The selected boy for the nth room is removed from the boys list as he has been allotted a room
        boys = list(set(boys) - set(double_room_dict_b[key]))
        # A separate list maintaining the list of double rooms which are only singley filled
        only_one_boy_in_double_room.append(key) 
        # The selected room is removed from the double_room_copy
        double_room_copy = list(set(double_room_copy) - set([key])) 

      # III. if more than 2 boys obtain the same double room number, then simply allott the room to the 1st two boys
      elif len(double_room_dict_b[key])>2:
        i=0
        for person in double_room_dict_b[key]:
          # Room Allocation
          room_allocation[person]=key
          i+=1
          if i==2:
            break
        
        # The selected boys for the nth room are removed from the boys list as they have been allotted a room
        boys = list(set(boys) - set(double_room_dict_b[key][0:2]))
        # The selected room is removed from the double_room_copy as it has been allotted to 2 boys
        double_room_copy = list(set(double_room_copy) - set([key]))

    ##########
    
    ### Single Room Allocation
    ## In single room allocation, boys are allocated before girls!
    # BOYS
    single_room_dict = defaultdict(list)

    # Seeing if the new positions obtained for boys have any value for single rooms, if there is, then appending it into single_room_dict where keys are the room numbers and values are the boys
    for boys_left in boys:
      if pos_vel[0][boys_left-1] in single_room_copy:
        single_room_dict[pos_vel[0][boys_left-1]].append(boys_left)

    for key in dict(single_room_dict):
      # I. if exact 1 boy obtain a single occupancy room number, then simply allott the room to him
      if len(double_room_dict[key])==1:
        for person in single_room_dict[key]:
          # Room Allocation
          room_allocation[person]=key
    
        # The selected boy for the nth room is removed from the boys list as he has been allotted a room
        boys = list(set(boys) - set(single_room_dict[key])) 
        # The selected room is removed from the single_room_copy as it has been allotted to a boy
        single_room_copy = list(set(single_room_copy) - set([key])) 

      # II. if more than one boy obtain the same single room number, then simply allott the room to the 1st boy
      elif len(single_room_dict[key])>1:
        i=0
        for person in single_room_dict[key]:
          # Room Allocation
          room_allocation[person]=key
          i+=1
          if i==1:
            break
      
        # The selected boy for the nth room is removed from the boys list as he has been allotted a room
        boys = list(set(boys) - set(single_room_dict[key][0:1]))
        # The selected room is removed from the single_room_copy as it has been allotted to a boy
        single_room_copy = list(set(single_room_copy) - set([key]))


    # GIRLS
    single_room_dict = defaultdict(list) 

    # Seeing if the new positions obtained for boys have any value for single rooms, if there is, then appending it into single_room_dict where keys are the room numbers and values are the boys
    for girls_left in girls:
      if pos_vel[0][girls_left-1] in single_room_copy:
        single_room_dict[pos_vel[0][girls_left-1]].append(girls_left)

    for key in dict(single_room_dict):
      # I. if exact 1 girl obtain a single occupany room number, then simply allott the room to her
      if len(double_room_dict[key])==1:
        for person in single_room_dict[key]:
          # Room Allocation
          room_allocation[person]=key
        
        # The selected girl for the nth room is removed from the girls list as she has been allotted a single occupancy room 
        girls = list(set(girls) - set(single_room_dict[key])) 
        # The selected room is removed from the single_room_copy as it has been allotted to a girl
        single_room_copy = list(set(single_room_copy) - set([key])) 

      # II. if more than one girl obtain the same single room number, then simply allott the room to the 1st girl
      elif len(single_room_dict[key])>1:
        i=0
        for person in single_room_dict[key]:
          # Room Allocation
          room_allocation[person]=key
          i+=1
          if i==1:
            break
      
        # The selected girl for the nth room is removed from the girls list as she has been allotted a single occupancy room
        girls = list(set(girls) - set(single_room_dict[key][0:1]))
        # The selected room is removed from the single_room_copy as it has been allotted to a girl
        single_room_copy = list(set(single_room_copy) - set([key]))  


    ########## Miscellaneous

    ## Only one girl in double room has numbers of those double rooms which are singley filled by girls!
    # Before filling one more girl in all those singley filled double rooms, checking how many girls are left for allotment. If they are less than the number of girls required to fill those singley filled double rooms, then the 'while' loop is executed.
    # This 'while' loop removes the other girls (who occupied the double rooms) too if the number of the girls left are not enough to fill the same
    while len(girls)<len(only_one_girl_in_double_room):
      x_room = only_one_girl_in_double_room[0]
      x_girl = double_room_dict[x_room]
      # print(x_girl)
      x_girl = x_girl[0]
      room_allocation[x_girl] = 0
      double_room_copy.append(x_room)
      double_room_copy.sort()
      girls.append(x_girl)
      girls.sort()
      only_one_girl_in_double_room.pop(0)

    # Filling the half filled double rooms (Girls)
    if len(only_one_girl_in_double_room) > 0:
      selected_girls_for_double_room = sample(set(girls),len(only_one_girl_in_double_room)) # randomly selecting girls
      i=0
      for item in only_one_girl_in_double_room:
        # Room Allocation
        room_allocation[selected_girls_for_double_room[i]] = item
        i+=1

      # The selected girls for the nth room are removed from the girls list as they have been allotted rooms
      girls = list(set(girls) - set(selected_girls_for_double_room))


    ## Only one boy in double room has numbers of those double rooms which are singley filled by boys!
    # Before filling one more boy in all those singley filled double rooms, checking how many boys are left for allotment. If they are less than the number of boys required to fill those singley filled double rooms, then the 'while' loop is executed.
    # This 'while' loop removes the other boys (who occupied the double rooms) too if the number of the boys left are not enough to fill the same.
    while len(boys)<len(only_one_boy_in_double_room):
      x_room = only_one_boy_in_double_room[0]
      x_boy = double_room_dict_b[x_room]
      x_boy = x_boy[0]
      room_allocation[x_boy] = 0
      double_room_copy.append(x_room)
      double_room_copy.sort()
      boys.append(x_boy)
      boys.sort()
      only_one_boy_in_double_room.pop(0)

    # Filling the half filled double rooms (Boys)
    if len(only_one_boy_in_double_room) > 0: 
      selected_boys_for_double_room = sample(set(boys),len(only_one_boy_in_double_room))
      i=0
      for item in only_one_boy_in_double_room:
        # Room Allocation
        room_allocation[selected_boys_for_double_room[i]] = item 
        i+=1

      # The selected boys for the nth room are removed from the boys list as they have been allotted rooms
      boys = list(set(boys) - set(selected_boys_for_double_room))


    ########## 
    ## If all the DOUBLE ROOMS are not filled
     
    if len(double_room_copy)!=0:
      for i in range (len(double_room_copy)):
        # If the number of girls left is more than the number of boys left, then allot that room girls otherwise to boys
        if len(girls)>=len(boys):
          selected_girls_for_double_room = sample(set(girls),2)
          for j in range(2):
            room_allocation[selected_girls_for_double_room[j]] = double_room_copy[0]
          double_room_copy.pop(0)
          girls = list(set(girls) - set(selected_girls_for_double_room))

        else:
          selected_boys_for_double_room = sample(set(boys),2)
          for j in range(2):
            room_allocation[selected_boys_for_double_room[j]] = double_room_copy[0]
          double_room_copy.pop(0)
          boys = list(set(boys) - set(selected_boys_for_double_room))


    ##########
    # This is to FILL REST all the SINGLE ROOMS!
    # The rest of the boys are allotted randomly to the left single occupancy rooms
    selected_boys_for_single_room = sample(set(boys),len(boys))
    

    for item in range (len(selected_boys_for_single_room)):
      # print(selected_boys_for_single_room[item])
      # print(single_room_copy)
      room_allocation[selected_boys_for_single_room[item]] = single_room_copy[0]
      single_room_copy.pop(0)

    boys = list(set(boys) - set(selected_boys_for_single_room)) 
    

    # The rest of the girls are allotted randomly to the left single occupancy rooms
    selected_girls_for_single_room = sample(set(girls),len(girls))

    for item in range (len(selected_girls_for_single_room)):
      room_allocation[selected_girls_for_single_room[item]] = single_room_copy[0]
      single_room_copy.pop(0)

    girls = list(set(girls) - set(selected_girls_for_single_room)) 

  
    ##########
    pos_vel[0] = list(room_allocation.values())

    # Evaluating the fitness value
    which_person = 0
    fitness_value = 0 ## Summation of all the preferences
  
    for pos in pos_vel[0]:
      # summing up all the preferences of the rooms alloted to people
      fitness_value += int(preferences.iloc[which_person,pos])
      which_person += 1

    # putting the fitness value as the third column of the list/matrix 'all_particles'
    pos_vel[2] = fitness_value

    
    # Updating the Pbest
    # Pbest
    if pos_vel[2]<pos_vel[5]:
      pos_vel[3] = pos_vel[0]
      pos_vel[4] = pos_vel[1]
      pos_vel[5] = pos_vel[2]


  all_particles.sort(key=sortThird) ## Sorting the rows on the basis of the fitness value which is in the 3rd column of 'all_particles' matrix
  # Gbest
  if Gbest[2]>all_particles[0][2]:
    Gbest[0] = all_particles[0][0]
    Gbest[1] = all_particles[0][1]
    Gbest[2] = all_particles[0][2]

  print(Gbest)

  Gbest_pos = []

  for i in range(56):
    pos = Gbest[0][i]
    Gbest_pos.append(int(preferences.iloc[i,pos]))

  Gbest_fitness = Gbest[2]
  Gbest_pos_fit = [Gbest_pos, Gbest_fitness]
  Gbest_list.append(Gbest_pos_fit) #Keeping Track of the Gbest at every iteration

print(Gbest)

# Export this Gbest list as an .xlsx file and then, use MATLAB and this list to make the animated video!! 



[[26, 23, 5, 20, 21, 24, 22, 31, 27, 15, 35, 27, 16, 13, 32, 40, 5, 6, 19, 11, 1, 33, 34, 15, 35, 30, 36, 19, 8, 25, 7, 4, 29, 7, 9, 29, 25, 39, 39, 17, 37, 38, 10, 17, 9, 28, 28, 37, 18, 8, 38, 12, 2, 14, 3, 18], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 931]
[[26, 31, 7, 19, 19, 23, 29, 29, 27, 37, 35, 36, 18, 37, 33, 21, 15, 7, 27, 18, 32, 38, 15, 30, 38, 34, 35, 40, 10, 16, 3, 4, 11, 6, 12, 1, 17, 22, 9, 14, 13, 9, 17, 28, 24, 25, 25, 39, 5, 28, 39, 8, 20, 8, 5, 2], [0.0, 9.774120446718884, -10.516108008026173, 1.4936994913387516, -5.974797965355006, 5.538730431684363, -1.7792381033721159, 4.3257073680168885, -2.5814372333343156, -2.245411421370215, 0.0, -0.3506123650503632, -2.832007876678137, -8.62617113606841, -2.085158131008761, 11.51352839703937, -17.05712101308582, -1.4936994913387516, -6.877038817023748, -6.13505125571646, -4.481098474016255, 1.2984

In [ ]:
pip install xlsxwriter

     |████████████████████████████████| 153kB 2.5MB/s 


In [ ]:
import xlsxwriter

with xlsxwriter.Workbook('/content/drive/My Drive/SEMs-&-E-books/SEM-7/NIC/Gbest_list6.xlsx') as workbook:
  worksheet = workbook.add_worksheet()

  for row_num,data in enumerate(Gbest_list):
    worksheet.write_row(row_num, 0, data[0])
    # print(data)
    worksheet.write_row(row_num, 56, [data[1]])